## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-17-15-08-41 +0000,nypost,"Parades, crawfish and beads: New Orleans celeb...",https://nypost.com/2026/02/17/us-news/new-orle...
1,2026-02-17-15-05-57 +0000,bbc,Netflix and Paramount are battling for Warner ...,https://www.bbc.com/news/articles/cm21z4zgkx7o...
2,2026-02-17-15-03-14 +0000,bbc,US soldiers arrive in Nigeria to aid its fight...,https://www.bbc.com/news/articles/cj6d79gyrdro...
3,2026-02-17-14-58-44 +0000,nyt,Here’s more on his life.,https://www.nytimes.com/live/2026/02/17/us/jes...
4,2026-02-17-14-58-01 +0000,startribune,"The Rev. Jesse Jackson, who led the Civil Righ...",https://www.startribune.com/the-rev-jesse-jack...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2566/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
36,trump,30
27,jackson,15
3,new,14
75,talks,14
135,iran,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
121,2026-02-17-09-48-00 +0000,wsj,"Jesse Jackson, the civil-rights activist and p...",https://www.wsj.com/politics/jesse-jackson-civ...,85
44,2026-02-17-13-26-00 +0000,nypost,"Iran’s supreme leader taunts Trump, US as high...",https://nypost.com/2026/02/17/us-news/irans-su...,81
52,2026-02-17-12-51-03 +0000,nypost,Warner Bros. Discovery will restart talks with...,https://nypost.com/2026/02/17/business/warner-...,72
126,2026-02-17-09-25-31 +0000,nyt,Trump Said He’d End the War in Ukraine in a Da...,https://www.nytimes.com/2026/02/17/world/europ...,71
242,2026-02-16-18-07-29 +0000,wapo,"Popular anger burns in Iran after crackdown, a...",https://www.washingtonpost.com/world/2026/02/1...,65


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
121,85,2026-02-17-09-48-00 +0000,wsj,"Jesse Jackson, the civil-rights activist and p...",https://www.wsj.com/politics/jesse-jackson-civ...
44,78,2026-02-17-13-26-00 +0000,nypost,"Iran’s supreme leader taunts Trump, US as high...",https://nypost.com/2026/02/17/us-news/irans-su...
52,58,2026-02-17-12-51-03 +0000,nypost,Warner Bros. Discovery will restart talks with...,https://nypost.com/2026/02/17/business/warner-...
28,44,2026-02-17-14-00-30 +0000,nypost,Pulse-pounding video shows heroes wrestle tran...,https://nypost.com/2026/02/17/us-news/video-sh...
37,37,2026-02-17-13-40-58 +0000,nyt,Ukraine and Russia Hold New Round of Peace Tal...,https://www.nytimes.com/2026/02/17/world/europ...
251,36,2026-02-16-17-28-44 +0000,nypost,Kremlin slams claims it killed top Putin criti...,https://nypost.com/2026/02/16/world-news/kreml...
244,32,2026-02-16-17-54-58 +0000,nypost,French police raid Arab World Institute over f...,https://nypost.com/2026/02/16/world-news/frenc...
237,31,2026-02-16-18-25-57 +0000,nypost,Hegseth ‘close’ to blacklisting AI firm Anthro...,https://nypost.com/2026/02/16/business/hegseth...
103,30,2026-02-17-10-53-53 +0000,nypost,‘White Lotus’ star Natasha Rothwell goes off-s...,https://nypost.com/2026/02/17/us-news/white-lo...
58,30,2026-02-17-12-41-09 +0000,bbc,Nancy Guthrie's family cleared as possible sus...,https://www.bbc.com/news/articles/cjd93rl849xo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
